# HCAN 361 Final Project: Spring 2021

## The Nursing Workforce in US Hospitals: An Exploratory Analysis

- Focus on the Nursing workforce --\> Beds per nurse with logic that higher rates may have lower quality of service
- Though findings among nurse to patient rates effect on patient outcomes are inconclusive (journals.lww.com/jonajournal/Fulltext/2004/07000/Nurse\_Patient\_Ratios\_\_A\_Systematic\_Review\_on\_the.5.aspx), there is evidence suggesting that increased nurse-to-patient ratios is associated with nurse burnout and job satisfaction (sciencedirect.com/science/article/pii/S0029655417302658)
- Part of analysis is an informal sensitivty analysis of access given varying nurse to bed rates 

            - With this being exploratory, no definitive claims are made on what a good nurse to bed rate is. the point is to understand areas adversly impacted if thresholds are selected or increased

In [1]:
use [HCAN361_FinalProj]


--Identifying hospitals, staffed beds, and supply of nurses
--drop table #Nurse
SELECT A.ID
      ,cast(A.LAT as float) as LAT
	  ,cast(A.LONG as float) as LONG
	  ,A.MNAME
      ,C.County_Name,C.State_Name,C.HRSA_RURAL_DESC,('Z'+C.STATEFP+C.COUNTYFP) as [County_GEOID_Z]
	  ,case when C.[Median_ADI_NatRank]<=25 then '02: Upper Class'
	   when (C.[Median_ADI_NatRank]>25 and C.[Median_ADI_NatRank]<=75) then '01: Middle Class'
	   when C.[Median_ADI_NatRank]>75 then '00: Lower Class'
	   else '99: Not Ranked' end as [ADI]
	  ,(isnull(D.HOSPBD,0)-isnull(D.SNBD88,0)) as BEDS_Non_Nursing
	  ,D.HOSPBD as BEDS
	  ,(isnull(E.[FTRNTF],0)+(isnull(E.[PTRNTF],0)*0.50))as [Est_FTE_Nurses]
	  ,E.FTEN as [AHA Reported Nurse FTE]
INTO #Nurse
FROM 
dbo.AS19DEM as A
left join
[dbo].[Provider_Census_Tract] as B
	on A.ID=B.Provider_ID
left join 
[dbo].[US_Census_Tract_Charac] as C
	on B.GEOID_Data=C.GEOID_Data
left join
dbo.as19svc1 as D
	on D.ID=A.ID 
left join
dbo.as19util1 as E
	on A.ID=E.ID
where C.County_Name is not null --and (D.GENHOS='1' or D.PEDHOS='1')
order by C.State_Name;

--[HOSPBD] -->Total hospital beds (calculated)-->as19svc1
--[SNBD88] -->Skilled nursing care beds -->as19svc1
--[FTRNTF] -->Full-time registered nurses-->as19util1-->Full-time registered nurses
--[PTRNTF] -->Part-time registered nurses-->as19util1-->Part-time registered nurses
--[FTEN] -->Registered nurses FTE-->as19util1-->Registered nurses FTE (Reported)-->Problem with this measure is the number of nulls
--One limitation that may be worth mentioning is that secured hospitals like Taylor Hardin (i.e., court appointed mental facilities) are hard to filter out. Cannot find the column to remove facilities like that.

select top 10 * from #Nurse;

select distinct *
into HCAN361_25.dbo.Locs_Group2
from #Nurse;

alter table HCAN361_25.dbo.Locs_Group2
alter column ID varchar(30) not null;

alter table HCAN361_25.dbo.Locs_Group2
add constraint pk_nurs_Grp2 primary key(ID);

(6152 rows affected)

(10 rows affected)

(6152 rows affected)

Total execution time: 00:00:00.488

ID,LAT,LONG,MNAME,County_Name,State_Name,HRSA_RURAL_DESC,County_GEOID_Z,ADI,BEDS_Non_Nursing,BEDS,Est_FTE_Nurses,AHA Reported Nurse FTE
6660009,40.7949,-96.6237,Lincoln Surgical Hospital,Lancaster,Nebraska,00: Urban,Z31109,01: Middle Class,7,7,18.5,NULL
6660028,42.529,-98.9826,West Holt Medical Services,Holt,Nebraska,01: Rural,Z31089,01: Middle Class,17,17,30,NULL
6660135,41.1067,-98.0021,Merrick Medical Center,Merrick,Nebraska,01: Rural,Z31121,00: Lower Class,20,20,36.5,NULL
6660192,40.1558,-97.1749,Jefferson Community Health and Life,Jefferson,Nebraska,01: Rural,Z31095,01: Middle Class,17,57,37,38
6660298,40.8488,-101.727,Perkins County Health Services,Perkins,Nebraska,01: Rural,Z31135,01: Middle Class,20,20,25.5,NULL
6660340,40.5233,-101.65,Chase County Community Hospital,Chase,Nebraska,01: Rural,Z31029,01: Middle Class,20,20,19,NULL
6660395,40.7854,-99.7574,Lexington Regional Health Center,Dawson,Nebraska,01: Rural,Z31047,00: Lower Class,25,25,41,NULL
6660414,40.6825,-99.1126,Kearney Regional Medical Center,Buffalo,Nebraska,01: Rural,Z31019,01: Middle Class,92,92,89.5,NULL
6660435,40.7902,-96.6473,Madonna Rehabilitation Hospital,Lancaster,Nebraska,00: Urban,Z31109,01: Middle Class,308,308,399,NULL
6660560,41.1217,-100.77,Great Plains Health,Lincoln,Nebraska,01: Rural,Z31111,01: Middle Class,116,116,252,227


In [2]:
--Build Pop Tables for analyses
select A.[GEOID_Data], ('Z'+A.STATEFP+A.COUNTYFP) as GEOID_County_Z,
[State_Name],[County_Name], [HRSA_RURAL_DESC],
case when [Median_ADI_NatRank]<=25 then '02: Upper Class'
when ([Median_ADI_NatRank]>25 and [Median_ADI_NatRank]<=75) then '01: Middle Class'
when [Median_ADI_NatRank]>75 then '00: Lower Class'
else '99: Not Ranked' end as [ADI],B.B01001e1 as [Total_Pop]
into #Population
from [dbo].[US_Census_Tract_Charac] A
left join
[dbo].[Census_ACS5Yr_2018] B
	on A.GEOID_Data=B.GEOID_Data;

select top 10 *
from #Population

(74001 rows affected)

(10 rows affected)

Total execution time: 00:00:00.303

GEOID_Data,GEOID_County_Z,State_Name,County_Name,HRSA_RURAL_DESC,ADI,Total_Pop
14000US06037430801,Z06037,California,Los Angeles,00: Urban,02: Upper Class,6792.00000000
14000US06037430802,Z06037,California,Los Angeles,00: Urban,02: Upper Class,4534.00000000
14000US06037431001,Z06037,California,Los Angeles,00: Urban,02: Upper Class,7412.00000000
14000US06037431002,Z06037,California,Los Angeles,00: Urban,02: Upper Class,3120.00000000
14000US06037431100,Z06037,California,Los Angeles,00: Urban,02: Upper Class,7087.00000000
14000US06037431300,Z06037,California,Los Angeles,00: Urban,02: Upper Class,2722.00000000
14000US06037432602,Z06037,California,Los Angeles,00: Urban,02: Upper Class,4940.00000000
14000US06037432902,Z06037,California,Los Angeles,00: Urban,02: Upper Class,3380.00000000
14000US06037433200,Z06037,California,Los Angeles,00: Urban,02: Upper Class,6270.00000000
14000US06037433501,Z06037,California,Los Angeles,00: Urban,00: Lower Class,1870.00000000


In [3]:
 -- Minimum Travel Time to All Hospitals, Available Nurse FTE within 30 minutes, and Minimum Travel Time from a sensitivity analysis of Nurse to Beds Thresholds

--drop table #TravelMap
Select Distinct A.*
                ,ISNULL(B.Closest_Hospital,361) AS Closest_Hospital_As_Is
				,ISNULL(C.NurseFTE_30Min,0) AS NurseFTE_30Min
				,ISNULL(D.Closest_Hospital,361) AS Closest_Hospital_Above_PTile_10
				,ISNULL(E.Closest_Hospital,361) AS Closest_Hospital_Above_PTile_15
				,ISNULL(F.Closest_Hospital,361) AS Closest_Hospital_Above_PTile_20
into #TravelMap
from
--Population of interest
#Population A
left join
--Find closest hospital when all are open
(
select GEOID_Data, MIN(TOTAL_TRAVELTIME) AS Closest_Hospital
from 
[dbo].[Travel_Matrix]
where Provider_ID in
(select ID from #Nurse)
Group by GEOID_Data
) B
	on A.GEOID_Data=B.GEOID_Data
left join
--Find available Nurse FTE within 30 minutes
(
select A.GEOID_Data, SUM(B.Est_FTE_Nurses) AS NurseFTE_30Min
from 
[dbo].[Travel_Matrix] as A
left join
#Nurse B
	on A.Provider_ID=B.ID
where A.Provider_ID in
(select ID from #Nurse)
and A.TOTAL_TRAVELTIME<=30
Group by GEOID_Data
) C
	on A.GEOID_Data=C.GEOID_Data
--Find closest hospital when FTE/Beds<1 is not available
left join
(
select GEOID_Data, MIN(TOTAL_TRAVELTIME) AS Closest_Hospital
from 
[dbo].[Travel_Matrix]
where 
Provider_ID in
(select ID from #Nurse)
and
Provider_ID in
(select ID from #Nurse where (Est_FTE_Nurses/(BEDS_Non_Nursing*1.00))>=1)
Group by GEOID_Data
) D
	on A.GEOID_Data=D.GEOID_Data
--Find closest hospital when FTE/Beds<1.5 is not available
left join
(
select GEOID_Data, MIN(TOTAL_TRAVELTIME) AS Closest_Hospital
from 
[dbo].[Travel_Matrix]
where 
Provider_ID in
(select ID from #Nurse)
and
Provider_ID in
(select ID from #Nurse where (Est_FTE_Nurses/(BEDS_Non_Nursing*1.00))>=1.50)
Group by GEOID_Data
) E
	on A.GEOID_Data=E.GEOID_Data
--Find closest hospital when FTE/Beds<2 is not available
left join
(
select GEOID_Data, MIN(TOTAL_TRAVELTIME) AS Closest_Hospital
from 
[dbo].[Travel_Matrix]
where 
Provider_ID in
(select ID from #Nurse)
and
Provider_ID in
(select ID from #Nurse where (Est_FTE_Nurses/(BEDS_Non_Nursing*1.00))>=2)
Group by GEOID_Data
) F
	on A.GEOID_Data=F.GEOID_Data
order by A.GEOID_Data;

select top 10 * from #TravelMap;

select distinct *
into HCAN361_25.dbo.TravelMap_Group2
from #TravelMap;

alter table HCAN361_25.dbo.TravelMap_Group2
alter column GEOID_Data varchar(30) not null;

alter table HCAN361_25.dbo.TravelMap_Group2
add constraint pk_trvl_Grp2 primary key(GEOID_Data);


(74001 rows affected)

(10 rows affected)

(74001 rows affected)

Total execution time: 00:00:51.476

GEOID_Data,GEOID_County_Z,State_Name,County_Name,HRSA_RURAL_DESC,ADI,Total_Pop,Closest_Hospital_As_Is,NurseFTE_30Min,Closest_Hospital_Above_PTile_10,Closest_Hospital_Above_PTile_15,Closest_Hospital_Above_PTile_20
14000US19049050600,Z19049,Iowa,Dallas,00: Urban,01: Middle Class,2705.00000000,28.927683180458512,72.5,28.927683180458512,28.927683180458512,28.927683180458512
14000US19049050809,Z19049,Iowa,Dallas,00: Urban,01: Middle Class,9978.00000000,8.624315557187298,3548.5,17.956831870887456,17.956831870887456,17.977617142372207
14000US19049050811,Z19049,Iowa,Dallas,00: Urban,02: Upper Class,12508.00000000,8.827041612001112,3509,15.93671916898836,15.93671916898836,15.95750444047311
14000US19049050901,Z19049,Iowa,Dallas,00: Urban,01: Middle Class,3687.00000000,9.80090537495819,3509,21.923446201672007,21.923446201672007,21.944231473156755
14000US19049050902,Z19049,Iowa,Dallas,00: Urban,01: Middle Class,4013.00000000,16.869637287714426,3548.5,23.33448629982246,23.33448629982246,23.355271571307206
14000US19051080200,Z19051,Iowa,Davis,01: Rural,01: Middle Class,4684.00000000,14.037810298124409,41,14.037810298124409,14.037810298124409,14.037810298124409
14000US19053960300,Z19053,Iowa,Decatur,01: Rural,01: Middle Class,2172.00000000,0.044978472585642,131,0.044978472585642,0.044978472585642,0.044978472585642
14000US19059451000,Z19059,Iowa,Dickinson,01: Rural,01: Middle Class,2708.00000000,9.514285637844731,150.5,9.514285637844731,9.514285637844731,9.514285637844731
14000US19061000100,Z19061,Iowa,Dubuque,00: Urban,00: Lower Class,3146.00000000,5.900115477341917,700.5,5.900115477341917,6.252752725318453,6.252752725318453
14000US19061000600,Z19061,Iowa,Dubuque,00: Urban,01: Middle Class,3279.00000000,5.637410049230497,700.5,5.637410049230497,5.637410049230497,5.637410049230497


In [ ]:
 -- Standardized rate of population per bed at County Level
SELECT DISTINCT 
 *
,(BEDS_Non_Nursing/Total_Pop)*1000 as Beds_per_1K_People
,case when BEDS_Non_Nursing>0 then
 (Est_FTE_Nurses/BEDS_Non_Nursing) 
 else 0 end as Nurse_FTE_per_Bed
into #county_level
FROM
(
select A.State_Name
      ,A.County_Name
	  ,A.GEOID_County_Z
	  ,isnull(sum(B.BEDS_Non_Nursing),0) as BEDS_Non_Nursing 
	  ,isnull(sum(A.Total_Pop),0) as Total_Pop
	  ,isnull(sum(B.Est_FTE_Nurses),0) as Est_FTE_Nurses
from 
#Population as A
left join
#Nurse as B
	on A.State_Name=B.State_Name and 
	   A.County_Name=B.County_Name
group by A.State_Name, A.County_Name, A.GEOID_County_Z
)A
order by State_Name, County_Name, GEOID_County_Z;


select top 10 *
from #county_level;

select distinct *
into HCAN361_25.dbo.county_level_Grp2
from #county_level;

alter table HCAN361_25.dbo.county_level_Grp2
alter column GEOID_County_Z varchar(30) not null;

alter table HCAN361_25.dbo.county_level_Grp2
add constraint pk_cnty_Grp2 primary key(GEOID_County_Z);


Warning: Null value is eliminated by an aggregate or other SET operation.

(3220 rows affected)

(10 rows affected)

(3220 rows affected)

Total execution time: 00:00:05.301

State_Name,County_Name,GEOID_County_Z,BEDS_Non_Nursing,Total_Pop,Est_FTE_Nurses,Beds_per_1K_People,Nurse_FTE_per_Bed
Alabama,Barbour,Z01005,423,25782.00000000,432,16.40679543867815,1.0212765957446808
Alabama,Bibb,Z01007,140,22527.00000000,132,6.214764504816443,0.9428571428571428
Alabama,Cherokee,Z01019,270,25853.00000000,177,10.443662244226974,0.6555555555555556
Alabama,Choctaw,Z01023,100,13075.00000000,120,7.648183556405353,1.2
Alabama,Conecuh,Z01035,220,12514.00000000,155,17.58031005274093,0.7045454545454546
Alabama,Dale,Z01045,966,49255.00000000,1554,19.612222109430515,1.608695652173913
Alabama,Henry,Z01067,0,17124.00000000,0,0,0
Alabama,Jefferson,Z01073,712147,9238488.00000000,1669120,77.08480002355364,2.3437857633325705
Alabama,Lamar,Z01075,0,13933.00000000,0,0,0
Alabama,Lawrence,Z01079,387,33171.00000000,499.5,11.66681740074161,1.2906976744186047


In [ ]:
--Table 1: Nationwide Access to Nurse FTEs and Implications of Access if Definition of Appropriate Nurse FTE to Bed Ratio is Defined 
SELECT DISTINCT *
FROM
(
Select Distinct
 State_Name
 ,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY NurseFTE_30Min) OVER (PARTITION BY State_Name) AS [Median Nurse FTE W_in 30 Min Statewide]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Closest_Hospital_As_Is) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time As Is Statewide]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Closest_Hospital_Above_PTile_10) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time Above 1 FTE Statewide]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Closest_Hospital_Above_PTile_15) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time Above 1.5 FTE Statewide]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Closest_Hospital_Above_PTile_20) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time Above 2 FTE Statewide]

,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then NurseFTE_30Min End) OVER (PARTITION BY State_Name) AS [Median Nurse FTE W_in 30 Min Urban]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then Closest_Hospital_As_Is End) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time As Is Urban]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then Closest_Hospital_Above_PTile_10 End) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time Above 1 FTE Urban]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then Closest_Hospital_Above_PTile_15 End) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time Above 1.5 FTE Urban]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then Closest_Hospital_Above_PTile_20 End) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time Above 2 FTE Urban]


,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then NurseFTE_30Min End) OVER (PARTITION BY State_Name) AS [Median Nurse FTE W_in 30 Min Rural]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then Closest_Hospital_As_Is End) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time As Is Rural]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then Closest_Hospital_Above_PTile_10 End) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time Above 1 FTE Rural]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then Closest_Hospital_Above_PTile_15 End) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time Above 1.5 FTE Rural]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then Closest_Hospital_Above_PTile_20 End) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time Above 2 FTE Rural]
from
HCAN361_25.dbo.TravelMap_Group2

union all

Select Distinct
 'United States' as State_Name
 ,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY NurseFTE_30Min) OVER () AS [Median Nurse FTE W_in 30 Min Statewide]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Closest_Hospital_As_Is) OVER () AS [Median Closest Travel Time As Is Statewide]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Closest_Hospital_Above_PTile_10) OVER () AS [Median Closest Travel Time Above 1 FTE Statewide]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Closest_Hospital_Above_PTile_15) OVER () AS [Median Closest Travel Time Above 1.5 FTE Statewide]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Closest_Hospital_Above_PTile_20) OVER () AS [Median Closest Travel Time Above 2 FTE Statewide]

,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then NurseFTE_30Min End) OVER () AS [Median Nurse FTE W_in 30 Min Urban]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then Closest_Hospital_As_Is End) OVER () AS [Median Closest Travel Time As Is Urban]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then Closest_Hospital_Above_PTile_10 End) OVER () AS [Median Closest Travel Time Above 1 FTE Urban]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then Closest_Hospital_Above_PTile_15 End) OVER () AS [Median Closest Travel Time Above 1.5 FTE Urban]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then Closest_Hospital_Above_PTile_20 End) OVER () AS [Median Closest Travel Time Above 2 FTE Urban]


,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then NurseFTE_30Min End) OVER () AS [Median Nurse FTE W_in 30 Min Rural]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then Closest_Hospital_As_Is End) OVER () AS [Median Closest Travel Time As Is Rural]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then Closest_Hospital_Above_PTile_10 End) OVER () AS [Median Closest Travel Time Above 1 FTE Rural]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then Closest_Hospital_Above_PTile_15 End) OVER () AS [Median Closest Travel Time Above 1.5 FTE Rural]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then Closest_Hospital_Above_PTile_20 End) OVER () AS [Median Closest Travel Time Above 2 FTE Rural]
from
HCAN361_25.dbo.TravelMap_Group2
) Z
order by State_Name;

--First 2 columns of table is straightforward. The point of changing the Nurse to Bed FTE standards is to understand areas most impacted if Ideal Service is Defined as a Hospital Having a Minimum Threshold 


(53 rows affected)

Total execution time: 00:00:17.080

State_Name,Median Nurse FTE W_in 30 Min Statewide,Median Closest Travel Time As Is Statewide,Median Closest Travel Time Above 1 FTE Statewide,Median Closest Travel Time Above 1.5 FTE Statewide,Median Closest Travel Time Above 2 FTE Statewide,Median Nurse FTE W_in 30 Min Urban,Median Closest Travel Time As Is Urban,Median Closest Travel Time Above 1 FTE Urban,Median Closest Travel Time Above 1.5 FTE Urban,Median Closest Travel Time Above 2 FTE Urban,Median Nurse FTE W_in 30 Min Rural,Median Closest Travel Time As Is Rural,Median Closest Travel Time Above 1 FTE Rural,Median Closest Travel Time Above 1.5 FTE Rural,Median Closest Travel Time Above 2 FTE Rural
Alabama,973.5,11.601299666024637,12.695146486348929,15.71076673685974,22.19305385257254,2253,10.326237108115452,11.161877699350457,12.752160704678502,15.226566479950762,158.5,14.916436449722534,16.34386863853664,25.360394793653796,44.917420453141744
Alaska,157,16.648284081887677,17.66732048417082,19.501023958137928,23.24094938680561,1872,7.474293725853453,9.921465435284967,9.921465435284967,12.2461553745307,0,79.93941475465067,104.36259743592818,128.23016399845713,272.4036308539643
Arizona,5496.5,8.037048681338483,8.479878908754515,10.594907028237309,12.612827133709036,7134.5,7.328467667792145,7.869807309147876,9.772788605993757,11.865798333427989,31,29.524403768769826,29.524403768769826,36.16733113058223,46.982961191603366
Arkansas,465,12.82617281719356,14.863397458402755,19.662625290774336,36.315046298569186,1217.5,10.195383834820447,11.118568717221812,11.967673814468592,20.843322907576272,71.75,16.91061910809983,20.79046764044171,33.421002553679934,48.466315802722136
California,7135,7.612213342171019,8.075908911378981,8.80722425639523,10.265522264503339,7889,7.347234541828171,7.827608058842813,8.512382740212935,9.937200942864944,91,21.05377176247847,21.67266323331256,24.151278175144448,29.03608706179638
Colorado,2343.5,9.295610261031488,9.892381846678708,10.31316329483245,11.570915232308682,4948.5,8.624605267503092,9.16395632633345,9.688874187234589,10.8805987599135,38.5,26.2013803952933,29.611239298725465,31.191473798507182,32.61548328651295
Connecticut,4007,9.811690961128711,10.192832014988843,10.42442364351125,15.868590666833523,4813,9.507222716330482,10.041106635647983,10.207250606208087,15.786989838069882,512.5,15.497581781395528,15.497581781395528,15.497581781395528,18.187976468861276
Delaware,4732,10.456387967789286,12.121046684861575,12.212268597161735,13.477114884216455,4819,9.114570870965466,10.49564787127344,10.516505617477819,12.015666081252482,628,21.131438293939368,22.4832626961746,22.4832626961746,22.4832626961746
District of Columbia,11831.5,5.839016873359694,6.783119098385216,8.456008792058801,9.002115502579013,11831.5,5.839016873359694,6.783119098385216,8.456008792058801,9.002115502579013,NULL,NULL,NULL,NULL,NULL
Florida,3402,9.673219595604284,10.212234551498097,12.66134468232706,19.179492271277496,3972.75,9.471406123964389,9.933241876681553,12.354463529526987,18.54789937049756,113,20.010742496342694,21.551302758933446,28.081888649020662,42.022883109596386
